Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

---

In [66]:
# AZA MANAMPY CODE ATO FA MNAOVA CELLULE VAOVAO

from random import randrange
import numpy as np
from sklearn.metrics import mean_squared_error, log_loss
from sklearn.linear_model import HuberRegressor
from sklearn.datasets import load_boston, load_diabetes, load_iris, load_digits
from scipy.special import huber
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

def grad_check_sparse(f, x, analytic_grad, num_checks=12, h=1e-5, error=1e-9):
    """
    sample a few random elements and only return numerical
    in this dimensions
    """

    for i in range(num_checks):
        ix = tuple([randrange(m) for m in x.shape])

        oldval = x[ix]
        x[ix] = oldval + h  # increment by h
        fxph = f(x)  # evaluate f(x + h)
        x[ix] = oldval - h  # increment by h
        fxmh = f(x)  # evaluate f(x - h)
        x[ix] = oldval  # reset

        grad_numerical = (fxph - fxmh) / (2 * h)
        grad_analytic = analytic_grad[ix]
        rel_error = abs(grad_numerical - grad_analytic) / (
            abs(grad_numerical) + abs(grad_analytic)
        )
        print(
            "numerical: %f analytic: %f, relative error: %e"
            % (grad_numerical, grad_analytic, rel_error)
        )
        assert rel_error < error

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

# Robust linear regression - Huber loss


In [100]:
data = load_boston()
X_train1, y_train1 = data.data, data.target
w1 = np.random.randn(X_train1.shape[1]) * 0.0001
b1 = np.random.randn(1) * 0.0001

In [101]:
def huber_loss_naive(w, b, X, y, epsilon=1.35, alpha=0.0001):
    """
    Huber loss for all observations
    
    Inputs:
    - w: array of shape (D,) containing weights
    - b: float bias 
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels 
    - epsilon: float
    - alpha: regularization
    """
    loss = 0.0
    dw = np.zeros_like(w)
    db = 0.0
    
    # YOUR CODE HERE
    n = y.shape[0]
    #loss
    for i in range(n):
        if(abs(y[i] - (w.T@X[i] + b))<= epsilon):
            loss += (1/(2*n))*((w.T@X[i] + b - y[i])**2)
        else:
            loss += (1/n)*(epsilon*(abs((w.T@X[i] + b) - y[i]) - 0.5*(epsilon**2)))
    for i in range(w.shape[0]):
        loss += alpha * w[i]**2   
    
    #dw 
    for i in range(w.shape[0]):
        if(abs((w.T@X[i] + b) - y[i]) <= epsilon):
            for j in range(y.shape[0]):
                dw[i] += (1/n)*((w.T@X[j] - y[j] + b))*X[j][i] 
        else:
            if((w.T@X[i] + b) - y[i]<0):
                for j in range(y.shape[0]):
                    dw[i] -= (1/n)*(epsilon*X[j][i])
            elif(w.T@X[i] + b - y[i] > 0):
                for j in range(y.shape[0]):
                    dw[i] += (1/n)*(epsilon*X[j][i])
    for i in range(w.shape[0]):
        dw[i] += 2*alpha*w[i]
                    
    #db  
    for i in range(y.shape[0]):
        if(abs((w.T@X[i] + b) - y[i])<= epsilon):
            db += ((w.T@X[i] - y[i] + b))
        else:
            if((w.T@X[i] + b - y[i]) < 0):
                db -= epsilon
            elif((w.T@X[i] + b - y[i]) > 0):
                db += epsilon
    db = (1/n)*db
    
    return loss, dw, np.array(db).reshape(1,)


##### without regularization

In [102]:
loss, dw1, db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)

print("Gradient check w")
# Check with numerical gradient w
f = lambda w1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)[0]
grad_numerical = grad_check_sparse(f, w1, dw1, 15, error=1e-9)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)[0]
grad_numerical = grad_check_sparse(f2, b1, db1, 15, error=1e-9)


# Large epsilon
large_eps_loss, large_eps_dw1, large_eps_db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)

print("Gradient check w large epsilon")
# Check with numerical gradient w
f = lambda w1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)[0]
grad_numerical = grad_check_sparse(f, w1, large_eps_dw1, 15, error=1e-9)

print("Gradient check bias large epsilon")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)[0]
grad_numerical = grad_check_sparse(f2, b1, large_eps_db1, 15, error=1e-9)

Gradient check w
numerical: -17.081635 analytic: -17.081635, relative error: 8.349409e-11
numerical: -17.081635 analytic: -17.081635, relative error: 8.349409e-11
numerical: -15.034651 analytic: -15.034651, relative error: 1.563721e-11
numerical: -4.878257 analytic: -4.878257, relative error: 7.787857e-11
numerical: -8.484256 analytic: -8.484256, relative error: 3.140104e-11
numerical: -24.914970 analytic: -24.914970, relative error: 1.699002e-12
numerical: -15.034651 analytic: -15.034651, relative error: 1.563721e-11
numerical: -24.914970 analytic: -24.914970, relative error: 1.699002e-12
numerical: -15.034651 analytic: -15.034651, relative error: 1.563721e-11
numerical: -92.576117 analytic: -92.576117, relative error: 5.189221e-12
numerical: -24.914970 analytic: -24.914970, relative error: 1.699002e-12
numerical: -8.484256 analytic: -8.484256, relative error: 3.140104e-11
numerical: -551.120158 analytic: -551.120158, relative error: 8.021323e-13
numerical: -92.576117 analytic: -92.57

 ## with regularization

In [103]:
loss, dw1, db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)

print("Gradient check w")
# Check with numerical gradient w
f = lambda w1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)[0]
grad_numerical = grad_check_sparse(f, w1, dw1, 15, error=1e-9)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)[0]
grad_numerical = grad_check_sparse(f2, b1, db1, 15, error=1e-9)


# Large epsilon
large_eps_loss, large_eps_dw1, large_eps_db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)

print("Gradient check w large epsilon")
# Check with numerical gradient w
f = lambda w1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)[0]
grad_numerical = grad_check_sparse(f, w1, large_eps_dw1, 15, error=1e-9)

print("Gradient check bias large epsilon")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)[0]
grad_numerical = grad_check_sparse(f2, b1, large_eps_db1, 15, error=1e-9)

Gradient check w
numerical: -4.878419 analytic: -4.878419, relative error: 8.363409e-11
numerical: -0.748573 analytic: -0.748573, relative error: 8.997874e-10
numerical: -8.484288 analytic: -8.484288, relative error: 3.180375e-11
numerical: -15.034730 analytic: -15.034730, relative error: 1.261212e-11
numerical: -24.914892 analytic: -24.914892, relative error: 2.856300e-12
numerical: -0.748573 analytic: -0.748573, relative error: 8.997874e-10
numerical: -551.120353 analytic: -551.120353, relative error: 8.217289e-13
numerical: -15.034730 analytic: -15.034730, relative error: 1.261212e-11
numerical: -0.748573 analytic: -0.748573, relative error: 8.997874e-10
numerical: -0.094001 analytic: -0.094001, relative error: 2.918148e-09


AssertionError: 

In [129]:
def huber_loss_vectorized(w, b, X, y, epsilon=1.35, alpha=0.0001):
    """
    Huber loss for all observations
    
    Inputs:
    - w: array of shape (D,) containing weights
    - b: float bias 
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels 
    - epsilon: float
    - alpha: regularization
    """
    loss = 0.0
    dw = np.zeros_like(w)
    db = 0
    
    n = y.shape[0] #Number of samples
    f = (X @ w +b) - y
    tmp = np.copy(f)
    
    #loss
    c_inf = np.abs(f) <= epsilon
    c_sup = np.abs(f) > epsilon
    
    loss_1 = f**2/2
    loss_2 = epsilon  * np.fabs(f) - epsilon**2/2
    
    tmp[c_inf] = loss_1[c_inf]
    tmp[c_sup] = loss_2[c_sup]
    
    loss = (1/n) * np.sum(tmp) + alpha * (w.T @ w)
    
    #dw  
    loss_1 = f
    loss_2 = epsilon*(np.sign(f))
    
    tmp[c_inf] = loss_1[c_inf]
    tmp[c_sup] = loss_2[c_sup]
     
    dw = (1/n)*(X.T @ tmp) + 2*alpha*w
   
    #db
    db = (1/n) * np.sum(tmp)
    
    return loss, dw, np.array(db).reshape(1,)
     # YOUR CODE HERE
   """ loss = 0.0
    dw = np.zeros_like(w)
    db = 0
    
    f = (X@w +b -y)
    tmp = f.copy(tmp)
    #loss
    c1 = np.abs(f) <= epsilon
    c2 = np.abs(f) > epsilon
    
    loss1 = (f**2)/2
    loss2 = epsilon * np.abs(f) - (epsilon**2)/2
    
    tmp[c1] = loss1[c1]
    tmp[c2] = loss2[c2]
    
    loss =  np.sum(tmp) + alpha*(w.T@w)
    
    #dw
    loss1 = f
    loss2 = epsilon*np.sign(f)
    
    tmp[c1] = loss1[c1]
    tmp[c2] = loss2[c2]
   """ 
   
    
    
    """return loss, dw, np.array(db).reshape(1,)"""



## without regularization

In [130]:
loss, dw1, db1 = huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)

print("Gradient check w")
# Check with numerical gradient w
f = lambda w1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)[0]
grad_numerical = grad_check_sparse(f, w1, dw1, 15, error=1e-9)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)[0]
grad_numerical = grad_check_sparse(f2, b1, db1, 15, error=1e-9)


# Large epsilon
large_eps_loss, large_eps_dw1, large_eps_db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)

print("Gradient check w large epsilon")
# Check with numerical gradient w
f = lambda w1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)[0]
grad_numerical = grad_check_sparse(f, w1, large_eps_dw1, 15, error=1e-9)

print("Gradient check bias large epsilon")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)[0]
grad_numerical = grad_check_sparse(f2, b1, large_eps_db1, 15, error=1e-9)

Gradient check w
numerical: -17.081635 analytic: -17.081635, relative error: 3.005375e-13
numerical: -92.576117 analytic: -92.576117, relative error: 5.658178e-13
numerical: -8.484256 analytic: -8.484256, relative error: 1.046426e-11
numerical: -12.891700 analytic: -12.891700, relative error: 8.038712e-13
numerical: -0.093379 analytic: -0.093379, relative error: 6.023259e-10
numerical: -8.484256 analytic: -8.484256, relative error: 1.046426e-11
numerical: -15.340909 analytic: -15.340909, relative error: 1.861069e-12
numerical: -12.891700 analytic: -12.891700, relative error: 8.038712e-13
numerical: -15.340909 analytic: -15.340909, relative error: 1.861069e-12
numerical: -4.878257 analytic: -4.878257, relative error: 5.050771e-12
numerical: -551.120158 analytic: -551.120158, relative error: 1.031416e-15
numerical: -0.093379 analytic: -0.093379, relative error: 6.023259e-10
numerical: -15.340909 analytic: -15.340909, relative error: 1.861069e-12
numerical: -92.576117 analytic: -92.576117

## with regularization

In [131]:
loss, dw1, db1 = huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)

print("Gradient check w")
# Check with numerical gradient w
f = lambda w1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)[0]
grad_numerical = grad_check_sparse(f, w1, dw1, 15, error=1e-9)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)[0]
grad_numerical = grad_check_sparse(f2, b1, db1, 15, error=1e-9)


# Large epsilon
large_eps_loss, large_eps_dw1, large_eps_db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)

print("Gradient check w large epsilon")
# Check with numerical gradient w
f = lambda w1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)[0]
grad_numerical = grad_check_sparse(f, w1, large_eps_dw1, 15, error=1e-9)

print("Gradient check bias large epsilon")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)[0]
grad_numerical = grad_check_sparse(f2, b1, large_eps_db1, 15, error=1e-9)

Gradient check w
numerical: -17.081660 analytic: -17.081660, relative error: 3.130993e-12
numerical: -4.878419 analytic: -4.878419, relative error: 1.080870e-11
numerical: -5.123266 analytic: -5.123266, relative error: 4.746732e-12
numerical: -92.576201 analytic: -92.576201, relative error: 1.545789e-13
numerical: -24.914892 analytic: -24.914892, relative error: 2.858082e-12
numerical: -17.081660 analytic: -17.081660, relative error: 3.130993e-12
numerical: -8.484288 analytic: -8.484288, relative error: 1.086704e-11
numerical: -92.576201 analytic: -92.576201, relative error: 1.545789e-13
numerical: -15.034730 analytic: -15.034730, relative error: 7.967461e-13
numerical: -4.878419 analytic: -4.878419, relative error: 1.080870e-11
numerical: -0.094001 analytic: -0.094001, relative error: 8.612960e-10
numerical: -551.120353 analytic: -551.120353, relative error: 1.856548e-14
numerical: -551.120353 analytic: -551.120353, relative error: 1.856548e-14
numerical: -17.081660 analytic: -17.0816

In [132]:
class LinearModel():
    def __init__(self):
        self.w = None
        self.b = None

    def train(self, X, y, learning_rate=1e-3, alpha=0.0001, num_iters=100, batch_size=200, verbose=False):
        N, d = X.shape
        
        if self.w is None: # Initialization
            self.w = 0.001 * np.random.randn(d)
            self.b = 0.0

        # Run stochastic gradient descent to optimize w
        
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None
                                                               
            # Sample batch_size elements in X_batch and y_batch
            # X_batch shape is  (batch_size, d) and y_batch shape is (batch_size,)                                                                                          
            # Hint: Use np.random.choice to generate indices
            # YOUR CODE HERE
            c = np.random.choice(N,batch_size,replace=False)
            X_batch = X[c,:]
            y_batch = y[c]
            
            # evaluate loss and gradient
            loss, dw, db = self.loss(X_batch, y_batch, alpha)
            loss_history.append(loss)

            # perform parameter update                                                                
            # Update the weights w and bias b using the gradient and the learning rate.          
            # YOUR CODE HERE
            self.w -= learning_rate*dw
            self.b -= learning_rate*db
            
            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))
                
        return loss_history

    def predict(self, X):
        pass

    def loss(self, X_batch, y_batch, reg):
        pass

class HuberRegression(LinearModel):
    """ Linear regression """

    def loss(self, X_batch, y_batch, alpha):
        return huber_loss_vectorized(self.w, self.b, X_batch, y_batch, alpha=alpha)
    
    def predict(self, X):
        # YOUR CODE HERE
        return X@self.w + self.b

In [133]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train1)

sk_model = HuberRegressor(fit_intercept=True)
sk_model.fit(X_train1, y_train1)
sk_pred = sk_model.predict(X_train1)
sk_mse = mean_squared_error(sk_pred, y_train1)

model = HuberRegression()
model.train(X_train1, y_train1, num_iters=75000, batch_size=64, learning_rate=1e-2, verbose=True)
pred = model.predict(X_train1)
mse = mean_squared_error(pred, y_train1)

print("MSE scikit-learn:", sk_mse)
print("MSE gradient descent model :", mse)
assert mse - sk_mse < 1

iteration 0 / 75000: loss 30.514156
iteration 10000 / 75000: loss 3.793361
iteration 20000 / 75000: loss 4.168876
iteration 30000 / 75000: loss 3.141731
iteration 40000 / 75000: loss 2.725681
iteration 50000 / 75000: loss 3.240582
iteration 60000 / 75000: loss 3.057136
iteration 70000 / 75000: loss 2.795261
MSE scikit-learn: 24.04102301055777
MSE gradient descent model : 24.42379055303109


# Multinomial logistic regression

In [134]:
data = load_iris()
X_train2, y_train2 = data.data, data.target

W = np.random.randn(X_train2.shape[1], 3) * 0.0001

In [136]:
def softmax_loss_naive(W, X, y, alpha):
    """
    Softmax loss function WITH FOR LOOPS

    Inputs:
    - W: array of shape (D, C) containing weights
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels
    - alpha: (float) regularization 

    Returns a tuple of:
    - loss as single float
    - gradient with respect to weights W;  same shape as W
    """
    
    # Initialization
    loss = 0.0
    dW = np.zeros_like(W)
    
    for i in range(num_train):
        z = X[i,:].dot(W)
        z = z - np.max(z)
        correct_z = z[y[i]]
        d = np.sum(np.exp(z))
        p = np.exp(correct_z)/d
        
    
    # Tandremo ny numeric instability
    # YOUR CODE HERE
    
    #https://tomaxent.com/2017/03/05/cs231n-Assignment-1-softmax/?fbclid=IwAR0Vqd25crcgQoswxlmS60W8lBnSapwJ2F5nZ3hh4EfJR1HDp3gBN_YB1sY

    num_train = X.shape[0]
    num_classes = W.shape[1]
    
    for i in range(num_train):
        z = X[i, :].dot(W)
        z -= np.max(z)
        correct_z = z[y[i]]
        d = np.sum(np.exp(z))
        p = np.exp(correct_z) / d
        loss += -np.log(p)
        
        for j in range(num_classes):
            if j == y[i]:
                dW[:, y[i]] += (np.exp(z[j]) / d - 1) * X[i, :]
            else:
                dW[:, j] += (np.exp(z[j]) / d) * X[i, :]
                
    loss /= num_train
    loss += 0.5 * alpha * np.sum(W * W)
    
    dW /= num_train
    dW += alpha * W

    return loss, dW

for i in range(num_train):
    z = X[i,:].dot(W)
    z -= np.max(z)
    c_z = z[y[i]]
    loss += -np/log(np.exp(c_z)/np.sum(np.exp(z)))

## Without regularization

In [137]:
loss, dW = softmax_loss_naive(W, X_train2, y_train2, 0.0)

f = lambda W: softmax_loss_naive(W, X_train2, y_train2, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-7)

numerical: 0.765581 analytic: 0.765581, relative error: 7.376027e-11
numerical: 0.028021 analytic: 0.028021, relative error: 2.421432e-10
numerical: 0.028021 analytic: 0.028021, relative error: 2.421432e-10
numerical: -0.247738 analytic: -0.247738, relative error: 4.943368e-10
numerical: -0.042410 analytic: -0.042410, relative error: 4.157753e-11
numerical: 0.095293 analytic: 0.095293, relative error: 2.307856e-10
numerical: -0.167921 analytic: -0.167921, relative error: 2.914256e-10
numerical: -0.031810 analytic: -0.031810, relative error: 3.978412e-09
numerical: -0.042410 analytic: -0.042410, relative error: 4.157753e-11
numerical: 0.028021 analytic: 0.028021, relative error: 2.421432e-10
numerical: -0.031810 analytic: -0.031810, relative error: 3.978412e-09
numerical: -0.031810 analytic: -0.031810, relative error: 3.978412e-09


## With regularization

In [138]:
loss, dW = softmax_loss_naive(W, X_train2, y_train2, 2)

f = lambda W: softmax_loss_naive(W, X_train2, y_train2, 2)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-7)

numerical: 0.028186 analytic: 0.028186, relative error: 1.896021e-10
numerical: 0.317729 analytic: 0.317729, relative error: 9.093787e-14
numerical: -0.042467 analytic: -0.042467, relative error: 1.361375e-10
numerical: 0.765350 analytic: 0.765350, relative error: 6.803799e-11
numerical: 0.028186 analytic: 0.028186, relative error: 1.896021e-10
numerical: 0.765350 analytic: 0.765350, relative error: 6.803799e-11
numerical: -0.275336 analytic: -0.275336, relative error: 2.751394e-11
numerical: -0.032097 analytic: -0.032097, relative error: 3.974254e-09
numerical: -0.123192 analytic: -0.123192, relative error: 7.443210e-11
numerical: 0.279555 analytic: 0.279555, relative error: 3.940987e-10
numerical: 0.095298 analytic: 0.095298, relative error: 2.213235e-10
numerical: 0.279555 analytic: 0.279555, relative error: 3.940987e-10


In [139]:
def stable_softmax(z):
    exps = np.exp(z - np.max(z, axis=1, keepdims=True)) 
    return exps/np.sum(exps, axis=1, keepdims=True)

In [141]:
def softmax_loss_vectorized(W, X, y, alpha, fit_intercept=False):
    """
    Softmax loss function WITHOUT FOR LOOPS

    Inputs:
    - W: array of shape (D, C) containing weights
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels
    - alpha: (float) regularization 

    Returns a tuple of:
    - loss as single float
    - gradient with respect to weights W;  same shape as W
    """
    # Initialize the loss and gradient to zero.
    loss = 0.0
    dW = np.zeros_like(W)

    # YOUR CODE HERE
    tmp = -np.log(p[np.arange(N),y])
    loss = np.sum(tmp)/N + alpha*np.sum(W*W)
    
    #https://deepnotes.io/softmax-crossentropy#derivative-of-softmax
    
    N = X.shape[0]
    Z = X@W 
    p = stable_softmax(Z)
    
    #loss
    tmp = -np.log(p[np.arange(N),y])
    loss = np.sum(tmp)/N + alpha*np.sum(W*W) 
    
    #dW
    d = p.copy()
    d[np.arange(N),y] -= 1
    dW = (1/N)*(X.T@d) + 2 * alpha * W

    return loss, dW

## Without regularization

In [142]:
loss, dW = softmax_loss_vectorized(W, X_train2, y_train2, 0.0)

f = lambda W: softmax_loss_vectorized(W, X_train2, y_train2, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-7)

numerical: -0.275443 analytic: -0.275443, relative error: 1.888367e-11
numerical: -0.167921 analytic: -0.167921, relative error: 3.244832e-10
numerical: 0.317852 analytic: 0.317852, relative error: 1.081661e-11
numerical: -0.247738 analytic: -0.247738, relative error: 5.167438e-10
numerical: 0.028021 analytic: 0.028021, relative error: 7.483890e-10
numerical: -0.031810 analytic: -0.031810, relative error: 3.978416e-09
numerical: -0.123314 analytic: -0.123314, relative error: 1.017584e-10
numerical: -0.167921 analytic: -0.167921, relative error: 3.244832e-10
numerical: 0.095293 analytic: 0.095293, relative error: 1.725328e-10
numerical: 0.279547 analytic: 0.279547, relative error: 4.992367e-10
numerical: 0.765581 analytic: 0.765581, relative error: 5.925857e-11
numerical: 0.317852 analytic: 0.317852, relative error: 1.081661e-11


## With regularization

In [143]:
loss, dW = softmax_loss_vectorized(W, X_train2, y_train2, 2)

f = lambda W: softmax_loss_vectorized(W, X_train2, y_train2, 2)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-7)

numerical: -0.032385 analytic: -0.032385, relative error: 3.970175e-09
numerical: -0.123070 analytic: -0.123070, relative error: 1.098691e-10
numerical: -0.248005 analytic: -0.248005, relative error: 5.166791e-10
numerical: 0.279563 analytic: 0.279563, relative error: 4.875307e-10
numerical: -0.042525 analytic: -0.042525, relative error: 2.917093e-10
numerical: -0.168061 analytic: -0.168061, relative error: 3.076248e-10
numerical: -0.275229 analytic: -0.275229, relative error: 2.689804e-11
numerical: 0.279563 analytic: 0.279563, relative error: 4.875307e-10
numerical: -0.597889 analytic: -0.597889, relative error: 8.642592e-11
numerical: -0.123070 analytic: -0.123070, relative error: 1.098691e-10
numerical: 0.279563 analytic: 0.279563, relative error: 4.875307e-10
numerical: 0.765118 analytic: 0.765118, relative error: 6.231218e-11


## Gradient descent

In [154]:
class LinearModel():
    def __init__(self, fit_intercept=True):
        self.W = None
        self.fit_intercept = fit_intercept

    def train(self, X, y, learning_rate=1e-3, alpha=0, num_iters=100, batch_size=200, verbose=False):
        if self.fit_intercept:
            # YOUR CODE HERE
            X = np.concatenate((np.ones((X.shape[0],1)),X),axis=1)
            
        N, d = X.shape
        
        C = (np.max(y) + 1) 
        if self.W is None: # Initialization
            self.W = 0.001 * np.random.randn(d, C)

        # Run stochastic gradient descent to optimize W
        
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None
                                                               
            # Sample batch_size elements in X_batch and y_batch
            # X_batch shape is  (batch_size, d) and y_batch shape is (batch_size,)                                                                                          
            # Hint: Use np.random.choice to generate indices
            # YOUR CODE HERE
            choice = np.random.choice(N, batch_size, replace=False)

            X_batch = X[choice,:]
            y_batch = y[choice]
            
            # evaluate loss and gradient
            loss, dW = self.loss(X_batch, y_batch, alpha)
            loss_history.append(loss)

            # perform parameter update                                                                
            # Update the weights w using the gradient and the learning rate.          
            # YOUR CODE HERE
            self.W = self.W - learning_rate * dW
            
            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))
                
        return loss_history

    def predict(self, X):
        pass

    def loss(self, X_batch, y_batch, reg):
        pass

class MultinomialLogisticRegressor(LinearModel):
    """ Softmax regression """

    def loss(self, X_batch, y_batch, alpha):
        return softmax_loss_vectorized(self.W, X_batch, y_batch, alpha)
    
    def predict(self, X):
        """ 
        Inputs:
        - X: array of shape (N, D) 

        Returns:
        - y_pred: 1-dimensional array of length N, each element is an integer giving the predicted class 
        """
        # YOUR CODE HERE
        if(self.fit_intercept):
            X = np.concatenate((np.ones((X.shape[0],1)),X),axis=1)
            #self.W[0] = 0
            
        y_pred = []
        
        for i in range(X.shape[0]):   
            z = X[i] @self.W
            z = z - z.max()
            y_pred.append(np.argmax(np.exp(z)/sum(np.exp(z))))
        
        return y_pred

In [155]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train2 = scaler.fit_transform(X_train2)

sk_model = LogisticRegression(fit_intercept=False)
sk_model.fit(X_train2, y_train2)
sk_pred = sk_model.predict(X_train2)
sk_accuracy = accuracy_score(y_train2, sk_pred)

model = MultinomialLogisticRegressor(fit_intercept=False)
model.train(X_train2, y_train2, num_iters=75000, batch_size=64, learning_rate=1e-3, verbose=True)
pred = model.predict(X_train2)
model_accuracy = accuracy_score(y_train2, pred)

print("Accuracy scikit-learn:", sk_accuracy)
print("Accuracy gradient descent model :", model_accuracy)
assert sk_accuracy - model_accuracy < 0.01

iteration 0 / 75000: loss 1.097771
iteration 10000 / 75000: loss 0.364422
iteration 20000 / 75000: loss 0.297538
iteration 30000 / 75000: loss 0.441212
iteration 40000 / 75000: loss 0.321370
iteration 50000 / 75000: loss 0.306862
iteration 60000 / 75000: loss 0.351471
iteration 70000 / 75000: loss 0.314518
Accuracy scikit-learn: 0.86
Accuracy gradient descent model : 0.8666666666666667


In [156]:
sk_model = LogisticRegression(fit_intercept=True)
sk_model.fit(X_train2, y_train2)
sk_pred = sk_model.predict(X_train2)
sk_accuracy = accuracy_score(y_train2, sk_pred)

model = MultinomialLogisticRegressor(fit_intercept=True)
model.train(X_train2, y_train2, num_iters=75000, batch_size=64, learning_rate=1e-3, verbose=True)
pred = model.predict(X_train2)
model_accuracy = accuracy_score(y_train2, pred)

print("Accuracy scikit-learn:", sk_accuracy)
print("Accuracy gradient descent model :", model_accuracy)
assert sk_accuracy - model_accuracy < 0.02

iteration 0 / 75000: loss 1.098279
iteration 10000 / 75000: loss 0.300075
iteration 20000 / 75000: loss 0.268020
iteration 30000 / 75000: loss 0.247827
iteration 40000 / 75000: loss 0.171910
iteration 50000 / 75000: loss 0.138045
iteration 60000 / 75000: loss 0.124669
iteration 70000 / 75000: loss 0.142057
Accuracy scikit-learn: 0.9733333333333334
Accuracy gradient descent model : 0.96


# K-Nearest Neighbor

## Computing distances

In [157]:
data = load_digits()
X_train3, y_train3 = data.data, data.target
X_train3, X_test3, y_train3, y_test3 = train_test_split(X_train3, y_train3, test_size=0.33, random_state=2)

def get_distances_two_loops_with_norm(X_train, X_test):
    num_test = X_test.shape[0]
    num_train = X_train.shape[0]
    distances = np.zeros((num_test, num_train))
    for i in range(num_test):
        for j in range(num_train):
            distances[i, j] = np.linalg.norm(X_test[i] - X_train[j])
    return distances

In [158]:
from scipy.spatial import distance

In [170]:
def get_distances_two_loops(X_train, X_test):
    """
    Compute the distance between each test point in X_test and each training point
    in X_train 

    Inputs:
    - X_test: array of shape (num_test, D) 

    Returns:
    - distances: array of shape (num_test, num_train), dists[i, j] is Euclidean distance between 
    the ith test point and the jth training point.
    """
    num_test = X_test.shape[0]
    num_train = X_train.shape[0]
    distances = np.zeros((num_test, num_train))
    for i in range(num_test):
        for j in range(num_train):
            # Ataovy ao anaty distances[i, j] ny distance entre ith test point sy th training point
            # Aza manao boucle instony ato anatiny
            # TSY MAHAZO MAMPIASA np.linalg.norm() :D
            # YOUR CODE HERE
            distances[i, j] = np.sqrt((X_test[i] - X_train[j]).T@(X_test[i] - X_train[j]))
            #distances[i,j] = distance.euclidean(X_test[i],X_train[j])
    return distances


In [171]:
distances = get_distances_two_loops(X_train3, X_test3)
true_distances = get_distances_two_loops_with_norm(X_train3, X_test3)

difference = np.linalg.norm(distances - true_distances, ord='fro')

print(difference)
assert difference < 1e-10

0.0


In [182]:
def compute_distances_one_loop(X_train, X_test):
    """
    Compute the distance between each test point in X_test and each training point
    in X_train 

    Inputs:
    - X_test: array of shape (num_test, D) 

    Returns:
    - dists: array of shape (num_test, num_train), dists[i, j] is Euclidean distance between 
    the ith test point and the jth training point.
    """
    num_test = X_test.shape[0]
    num_train = X_train.shape[0]
    distances = np.zeros((num_test, num_train))
    for i in range(num_test):
        # Ataovy ao anaty dists[i, j] ny distance entre ith test point sy th training point
        # Aza manao boucle instony ato anatiny
        # TSY MAHAZO MAMPIASA np.linalg.norm() :D
        
        # YOUR CODE HERE
        #j_iterate(distances,i,0,X_test,X_train)
        distances[i,:] = np.sqrt(np.sum(((X_test[i,:] - X_train)**2),axis=1))
    return distances    


In [183]:
distances = compute_distances_one_loop(X_train3, X_test3)
true_distances = get_distances_two_loops_with_norm(X_train3, X_test3)

difference = np.linalg.norm(distances - true_distances, ord='fro')

print(difference)
assert difference < 1e-10

0.0


In [174]:
from sklearn.metrics import pairwise_distances

In [190]:
def get_distances_zero_loop(X_train, X_test):
    """
    Compute the distance between each test point in X_test and each training point
    in X_train 

    Inputs:
    - X_test: array of shape (num_test, D) 

    Returns:
    - distances: array of shape (num_test, num_train), dists[i, j] is Euclidean distance between 
    the ith test point and the jth training point.
    """
    num_test = X_test.shape[0]
    num_train = X_train.shape[0]
    distances = np.zeros((num_test, num_train))  
    # Ataovy ao anaty dists[i, j] ny distance entre ith test point sy th training point
    # Aza manao boucle instony
    # TSY MAHAZO MAMPIASA np.linalg.norm() NA FONCTIONS AO AMIN'NY SCIPY :D
            
    # YOUR CODE HERE
    
    distances = np.sqrt(-2 * np.dot(X_test, X_train.T) + np.sum(X_train**2,axis=1) + np.sum(X_test**2, axis=1)[:, np.newaxis])

    return distances

In [191]:
distances = get_distances_zero_loop(X_train3, X_test3)
true_distances = get_distances_two_loops_with_norm(X_train3, X_test3)

difference = np.linalg.norm(distances - true_distances, ord='fro')

print(difference)
assert difference < 1e-10

0.0


## K-Nearest Neighbor (knn) classifier

In [177]:
from collections import Counter

In [178]:
class KNearestNeighborClassifier():
    """ kNN classifier using L2 distance """

    def __init__(self, k=1):
        """
        Inputs:
        - k: number of nearest neighbors that vote for the predicted labels.
        """
        self.k = k

    def fit(self, X, y):
        """
        Train the classifier. Just memorize the training data.

        Inputs:
        - X: array of shape (num_train, D) 
        - y: array of shape (N,) 
        """
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        """
        Predict labels for test data using this classifier.

        Inputs:
        - X: array of shape (num_test, D) 

        Returns:
        - y: array of shape (num_test,) 
        """
        distances = get_distances_zero_loop(self.X_train, X)
        return self.predict_labels(distances)

    def predict_labels(self, distances):
        """
        Given a matrix of distances between test points and training points,
        predict a label for each test point.

        Inputs:
        - distances: array of shape (num_test, num_train), dists[i, j] is Euclidean distance between 
        the ith test point and the jth training point.

        Returns:
        - y:  array of shape (num_test,) 
        """
        num_test = distances.shape[0]
        y_pred = np.zeros(num_test)
        for i in range(num_test):
            # list storing the labels of the k nearest neighbors to the ith test point.
            closest_y = []

            # Ampidirina ao anaty closest_y ny labels an'ny k neighbors akaiky indrindra
            # Jereo fampiasana np.argsort
            
            #https://github.com/MautKaFarishta/Simple-Data-Scince-Projects/blob/master/ML_From_Scratch/KNN/KNN.py
                
            dist = np.argsort(distances[i])[:self.k]
            closest_y = [ self.y_train[j] for j in dist ]

            # Tadiavo ny label betsaka indrindra dia iny no atao prediction
            # Raha misy mitovy dia izay label kely raisina
            
            #y_pred[i] = (Counter(closest_y).most_common()[0][0])
            y_pred[i] = np.argmax(np.bincount(closest_y))

        return y_pred
    """Accuracy scikit-learn: 0.9831649831649831
Accuracy gradient descent model : 0.9831649831649831"""

In [179]:
sk_model = KNeighborsClassifier(n_neighbors=3)
sk_model.fit(X_train3, y_train3)
sk_pred = sk_model.predict(X_test3)
sk_accuracy = accuracy_score(y_test3, sk_pred)

model = KNearestNeighborClassifier(k=3)
model.fit(X_train3, y_train3)
pred = model.predict(X_test3)
model_accuracy = accuracy_score(y_test3, pred)

print("Accuracy scikit-learn:", sk_accuracy)
print("Accuracy gradient descent model :", model_accuracy)
assert sk_accuracy - model_accuracy < 1e-10

Accuracy scikit-learn: 0.9831649831649831
Accuracy gradient descent model : 0.9831649831649831


## cross-validation

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [180]:
num_folds = 5
k_choices = [1, 3, 5, 8, 10, 12, 15, 20, 50, 100]

X_train_folds = []
y_train_folds = []

# Split up the data into folds
# X_train_folds and y_train_folds lits of length num_folds
# YOUR CODE HERE

#https://github.com/suryanktiwari/Linear-Regression-and-K-fold-Cross-Validation/blob/master/K%20fold%20Cross%20Validation.py
start = 0
i = 1
step = int(len(X_train3)/num_folds)

while i<=num_folds:
    X_train_folds.append(X_train3[start*step:i*step])
    y_train_folds.append(y_train3[start*step:i*step])
    start=i
    i+=1

# A dictionary of length num_folds holding the accuracies for different values of k 
k_to_accuracies = {}

# Ataovy ary ilay k-fold cross validation 
# Atao ao anaty k_to_accuracies ny accuracy isaky ny valeur k
# YOUR CODE HERE

for k in k_choices:
    cur = 0
    while cur<num_folds:
        model_accuracy = 0
        test_x = X_train_folds[cur]
        test_y = y_train_folds[cur]

        trainX = []
        trainY = []
        for xt,yt in zip(X_train_folds,y_train_folds):
            if (xt != test_x ).any(): 
                if(len(trainX) == 0):
                    trainX = xt
                else:
                    trainX = np.concatenate((trainX,xt),axis=0)

            if (yt != test_y).any():
                if(len(trainY) == 0):
                    trainY = yt
                else:
                    trainY = np.concatenate((trainY,yt),axis=0)

        model = KNearestNeighborClassifier(k=k)
        trainX = np.array(trainX)
        trainY = np.array(trainY)

        model.fit(trainX, trainY)
        pred = model.predict(test_x)

        model_accuracy = accuracy_score(test_y, pred)
        if(cur == 0):
            k_to_accuracies[k] = [model_accuracy]
        else:
            k_to_accuracies[k].append(model_accuracy)
        cur += 1

In [181]:
for k in sorted(k_to_accuracies):
    for accuracy in k_to_accuracies[k]:
        print('k = %d, accuracy = %f' % (k, accuracy))

k = 1, accuracy = 0.991667
k = 1, accuracy = 0.983333
k = 1, accuracy = 0.983333
k = 1, accuracy = 0.970833
k = 1, accuracy = 0.995833
k = 3, accuracy = 0.987500
k = 3, accuracy = 0.983333
k = 3, accuracy = 0.991667
k = 3, accuracy = 0.970833
k = 3, accuracy = 0.983333
k = 5, accuracy = 0.991667
k = 5, accuracy = 0.983333
k = 5, accuracy = 0.991667
k = 5, accuracy = 0.979167
k = 5, accuracy = 0.983333
k = 8, accuracy = 0.979167
k = 8, accuracy = 0.970833
k = 8, accuracy = 0.987500
k = 8, accuracy = 0.966667
k = 8, accuracy = 0.983333
k = 10, accuracy = 0.983333
k = 10, accuracy = 0.970833
k = 10, accuracy = 0.979167
k = 10, accuracy = 0.966667
k = 10, accuracy = 0.975000
k = 12, accuracy = 0.983333
k = 12, accuracy = 0.975000
k = 12, accuracy = 0.979167
k = 12, accuracy = 0.966667
k = 12, accuracy = 0.979167
k = 15, accuracy = 0.983333
k = 15, accuracy = 0.962500
k = 15, accuracy = 0.983333
k = 15, accuracy = 0.966667
k = 15, accuracy = 0.962500
k = 20, accuracy = 0.966667
k = 20, accu